In [1]:
# !pip install transformers --user
# !pip install pandas --user
import gc
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
def empty_cache():
  gc.collect()
  torch.cuda.empty_cache()

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [5]:
# !pip install wandb
import wandb

In [6]:
device = 'cuda'

In [7]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.ementa = self.data.ementa
        self.inteiro_teor = self.data.inteiro_teor

    def __len__(self):
        return len(self.ementa)

    def __getitem__(self, index):
        inteiro_teor = str(self.inteiro_teor[index])
        inteiro_teor = ' '.join(inteiro_teor.split())

        ementa = str(self.ementa[index])
        ementa = ' '.join(ementa.split())

        source = self.tokenizer.batch_encode_plus([inteiro_teor], max_length=self.source_len, pad_to_max_length=True,return_tensors='pt', truncation=True)
        target = self.tokenizer.batch_encode_plus([ementa], max_length=self.summ_len, pad_to_max_length=True,return_tensors='pt', truncation=True)

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [8]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]
        
        empty_cache()
        
        if _ % 10 == 0:
            wandb.log({"Loss do treinamento": loss.item()})

        if _ % 500 == 0:
            print(f'Época: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        empty_cache()

In [9]:
import os
# os.environ['WANDB_API_KEY'] = 'key'

In [10]:
def init_wandb(project_name):
    wandb.init(project=project_name) 
    config = wandb.config
    config.TRAIN_BATCH_SIZE = 1
    config.VALID_BATCH_SIZE = 1
    config.TRAIN_EPOCHS = 8
    config.VAL_EPOCHS = 1 
    config.LEARNING_RATE = 1e-4
    config.MAX_LEN = 1024
    config.SUMMARY_LEN = 400

    return config

In [11]:
def init_training_pipeline(config):
    print('Iniciando pipeline...\n\n')

    model_name = 't5-small'
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    empty_cache()
    
    train_dataset = pd.read_csv('../../data/train/translated-train-400-2500-summarized-new.csv', encoding='utf-8', error_bad_lines=False, engine="python")
    train_dataset = train_dataset[['ementa','inteiro_teor']]
    print('Exemplo de textos:')
    print(train_dataset.head(), '\n\n')
    
    val_dataset = pd.read_csv('../../data/train/translated-validate-400-2500-summarized-new.csv', encoding='utf-8', error_bad_lines=False, engine="python")
    val_dataset = val_dataset[['ementa','inteiro_teor']]

    print(f'Dataset de treino: {train_dataset.shape}')
    print(f'Dataset de teste: {val_dataset.shape}')

    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    train_params = {'batch_size': config.TRAIN_BATCH_SIZE, 'shuffle': True, 'num_workers': 0}
    val_params = {'batch_size': config.VALID_BATCH_SIZE, 'shuffle': False, 'num_workers': 0}

    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)
    
    print('Instanciando modelo...')
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    model = model.to(device)

    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    empty_cache()

    wandb.watch(model, log="all")
    
    print('Inicializando Fine-Tuning utilizando o dataset de acórdãos...')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

        empty_cache()

    print('Treinamento concluído!\n\n')

    return tokenizer, model, val_loader

In [12]:
config = init_wandb('t5-sum400-translated-summarized')

ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: thiagocmoreira (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.1
wandb: Run data is saved locally in wandb/run-20201010_234256-21xmbf5h
wandb: Syncing run youthful-sea-1


In [13]:
%%time
tokenizer, model, val_loader = init_training_pipeline(config)

Iniciando pipeline...


Exemplo de textos:
                                              ementa  \
0  interlocutory appeal. Work accident. partial r...   
1  interlocutory appeal. innominate precautionary...   
2  declaration embargoes. addictions. inexistence...   
3  interlocutory appeal. timing. jurisprudential ...   
4  interlocutory appeal. illicit outsourcing. Pre...   

                                        inteiro_teor  
0  judgment 6th class acv / rp This document of i...  
1  the c o rd (1st class class) gmwoc / db seen, ...  
2  judgment 5th class emp / igr visa, reported an...  
3  the c o rd (6th class) gmacc / sc / jr / pv vi...  
4  judgment 6th class acv / acc visa, report and ...   


Dataset de treino: (10791, 2)
Dataset de teste: (2698, 2)
Instanciando modelo...
Inicializando Fine-Tuning utilizando o dataset de acórdãos...
Época: 0, Loss:  5.317605018615723
Época: 0, Loss:  3.7711844444274902
Época: 0, Loss:  2.475536823272705
Época: 0, Loss:  0.40150442719459534
É

In [14]:
open("predictions.csv","w+")

<_io.TextIOWrapper name='predictions.csv' mode='w+' encoding='UTF-8'>

In [40]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    source_texts = []
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['source_ids'].to(device, dtype=torch.long)
            y = data['target_ids'].to(device, dtype=torch.long)
            mask = data['source_mask'].to(device, dtype=torch.long)

            generated_ids = model.generate(
                input_ids=ids,
                attention_mask=mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=1.5,
                length_penalty=0.95,
                early_stopping=True
            )
            
            src_texts = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in ids]
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]

            if _ % 100 == 0 and _ > 0:
                print(f'Resumos: {_} gerados')
            
            source_texts.extend(src_texts)
            predictions.extend(preds)
            actuals.extend(target)

    return source_texts, predictions, actuals

In [41]:
def validade_and_save_predictions(val_epochs, tokenizer, model, val_loader):
    print('Gerando sumários utilizando o modelo no dataset de validação...')
    for epoch in range(val_epochs):
        source_texts, predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({ 'inteiro_teor': source_texts, 'ementa_original': actuals, 'resumo_gerado': predictions })
        final_df.to_csv('predictions.csv')
        print('CSV para análise gerado!')

In [42]:
%%time
validade_and_save_predictions(config.VAL_EPOCHS, tokenizer, model, val_loader)

Gerando sumários utilizando o modelo no dataset de validação...
Resumos: 100 gerados
Resumos: 200 gerados
Resumos: 300 gerados
Resumos: 400 gerados
Resumos: 500 gerados
Resumos: 600 gerados
Resumos: 700 gerados
Resumos: 800 gerados
Resumos: 900 gerados
Resumos: 1000 gerados
Resumos: 1100 gerados
Resumos: 1200 gerados
Resumos: 1300 gerados
Resumos: 1400 gerados
Resumos: 1500 gerados
Resumos: 1600 gerados
Resumos: 1700 gerados
Resumos: 1800 gerados
Resumos: 1900 gerados
Resumos: 2000 gerados
Resumos: 2100 gerados
Resumos: 2200 gerados
Resumos: 2300 gerados
Resumos: 2400 gerados
Resumos: 2500 gerados
Resumos: 2600 gerados
CSV para análise gerado!
CPU times: user 3h 46min 36s, sys: 2min 13s, total: 3h 48min 50s
Wall time: 1h 39min 53s


In [49]:
generated_summaries = pd.read_csv('predictions.csv', encoding='utf-8', error_bad_lines=False, engine="python")
for index, row in generated_summaries[50:80].iterrows():
    print(f'Exemplo {index}', '\n')
    print('Ementa original:', row['inteiro_teor'], '\n\n')
    print('Ementa original:', row['ementa_original'], '\n\n')
    print('Sumário gerado:', row['resumo_gerado'], '\n\n')

Exemplo 50 

Ementa original: judgment 4th class gmfeo / igr viewed, reported and discussed these records of interlocutory appeal in magazine resource n ° tst-airr-1013 / 2005-006-23-41., and paulo cruz rodrigues and the supplementary pension fund of the bank of the amazon s.a. - the aggravated ones did not present a counter-draft to the interlocutory appeal or any reason for reviewing the appeal. the records were not sent to the public labor ministry. the appellant alleges that his review appeal is timely. therefore, the term started on 08/01/06 and ended on 08/08/2006. so the appeal filed on 08/08/2006, the last day of the deadline and not the day before the deadline, is perfectly timely (page 5, emphasis in the original). it is noted that the document presented on page 11, for the purpose of proving the publication of the judgment, is not a copy of the official diary or the electronic justice diary, which is why it has no value for the purpose sought by the party. 


Ementa original

In [44]:
# !pip install rouge-score tqdm --user
from tqdm import tqdm_notebook
# import time
from rouge_score import rouge_scorer, scoring
from typing import List, Dict

In [45]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [46]:
rouge_keys = ["rouge1", "rouge2", "rougeL"]
def calculate_rouge(output_lns: List[str], reference_lns: List[str]) -> Dict:
    scorer = rouge_scorer.RougeScorer(rouge_keys, use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()

    for reference_ln, output_ln in tqdm_notebook(zip(reference_lns, output_lns)):
        scores = scorer.score(reference_ln, output_ln)
        aggregator.add_scores(scores)

    result = aggregator.aggregate()
    return {k: v.mid.fmeasure for k, v in result.items()}

In [47]:
metrics = calculate_rouge(generated_summaries['ementa_original'], generated_summaries['resumo_gerado'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [48]:
metrics

{'rouge1': 0.498417746437988,
 'rouge2': 0.3180278120173391,
 'rougeL': 0.37623038416437526}

In [51]:
model_archive_name = 't5-model-english'
print('Salvando modelo treinado...')
model.save_pretrained(model_archive_name)
print(f'Modelo salvo na pasta {model_archive_name}!\n')

Salvando modelo treinado...
Modelo salvo na pasta t5-model-english!

